# Lab Segmentation 2

In [ ]:
from commonfunctions import *
%matplotlib inline
%load_ext autoreload
%autoreload 2
from skimage.color import rgb2gray
from skimage.color import rgba2rgb
from skimage.exposure import rescale_intensity

In [ ]:
# experiment 1


# img is gray scale
def getThreshold(img):

    if img.ndim == 3 and (img.shape)[2] == 3:
        img = rgb2gray(img)
        img = rescale_intensity(img, in_range=(0, 1.0), out_range=(0, 255))
        img = img.astype(np.uint8)
    elif img.ndim == 3 and (img.shape)[2] == 4:
        img = rgba2rgb(img)
        img = rgb2gray(img)
        img = rescale_intensity(img, in_range=(0, 1.0), out_range=(0, 255))
        img = img.astype(np.uint8)
        
    # calculate histogram
    img_flat = img.flatten()
    count = np.zeros(256, np.uint32)
    for pixel in img_flat:
        count[pixel] = count[pixel] + 1

    # get cdf for histogram and pixels' grayscale
    cdf = np.cumsum(count)

    gs_count = np.zeros(256, np.uint32)
    for gs in range(256):
        gs_count[gs] = gs * count[gs]
    gs_cdf = np.cumsum(gs_count)
    
    t_init = np.uint8(round(gs_cdf[255] / cdf[255]) if cdf[255] != 0 else 0)
    l_mean = gs_cdf[t_init] / cdf[t_init] if cdf[t_init] != 0 else 0  
    
    h_mean =  (gs_cdf[255] - gs_cdf[t_init]) / (cdf[255] - cdf[t_init]) if (cdf[255] - cdf[t_init]) != 0 else 0 
    
    new_t = np.uint8(round((l_mean + h_mean) / 2))
    
    old_t = t_init
    while old_t != new_t:
        old_t = new_t

        l_mean = gs_cdf[old_t] / cdf[old_t] if cdf[old_t] != 0 else 0 

        h_mean =  (gs_cdf[255] - gs_cdf[old_t]) / (cdf[255] - cdf[old_t]) if (cdf[255] - cdf[old_t]) != 0 else 0 
    
        new_t = np.uint8(round((l_mean + h_mean) / 2))

    return new_t



In [ ]:
for img_name in ["book.png", "book1.png", "cameraman.png", "cufe.png"]:
    img = io.imread(img_name)
    
    th = getThreshold(img)
    
    if img.ndim == 3 and (img.shape)[2] == 3:
        img = rgb2gray(img)
        img = rescale_intensity(img, in_range=(0, 1.0), out_range=(0, 255))
        img = img.astype(np.uint8)
    elif img.ndim == 3 and (img.shape)[2] == 4:
        img = rgba2rgb(img)
        img = rgb2gray(img)
        img = rescale_intensity(img, in_range=(0, 1.0), out_range=(0, 255))
        img = img.astype(np.uint8)
        
    new_img = img > th

    sectors = 4 # squares only
    new_img_local = np.copy(img)
    height = img.shape[0] // int(np.sqrt(sectors))
    width = img.shape[1] // int(np.sqrt(sectors))
    for sectorH in range(sectors//2):
        for sectorW in range(sectors//2):
            sub_img = img[height*sectorH:height*(sectorH+1), width*sectorW:width*(sectorW+1)]
            th = getThreshold(sub_img)
            
            sub_img = (sub_img > th) * 255
            
            new_img_local[height*sectorH:height*(sectorH+1), width*sectorW:width*(sectorW+1)] = sub_img

                
    show_images([img, new_img, new_img_local], ["original", "global_th", "local_th"])
    